## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

## Bring in our two files to join for time comparison.  

We want to look at the change in population per age group, as well as share of total population per age group, between the two five year periods represented by the American Community Survey 5-Year estimates. Here that's 2010-2014, and 2015-2019. We're going to bring in the 2014 file and rename the columns so that it's clear which is the older set - but I'm not going to specify the year so that this code can be reused with minimal changes when the next 5 year estimates come out. I'm going to bring in the 2019 csv as well. We can create a geodataframe later if we want.

In [24]:
# 2019 shapefile
new = pd.read_csv('../output/csv/agegroupsnogender_2019_cbsa.csv')

In [25]:
#2014 csv
old = pd.read_csv('../output/csv/agegroupsnogender_2014_cbsa.csv')

In [26]:
new.head()

CBSA           GEOID    total     U5  school  \
0      Big Stone Gap, VA Micro Area  310M500US13720    42456   2214    6189   
1           Billings, MT Metro Area  310M500US13740   179071  10811   30624   
2         Binghamton, NY Metro Area  310M500US13780   241874  12580   35319   
3  Birmingham-Hoover, AL Metro Area  310M500US13820  1085330  67638  182860   
4           Bismarck, ND Metro Area  310M500US13900   127503   8528   20643   

   18_20s     30s     40s   50_65     O65  PU5  Pschool  P18_20s  P30s  P40s  \
0    6933    5337    5350    8976    7457  5.2     14.6     16.3  12.6  12.6   
1   25857   23260   21387   36130   31002  6.0     17.1     14.4  13.0  11.9   
2   44875   26009   25965   51415   45711  5.2     14.6     18.6  10.8  10.7   
3  168169  144073  140593  213217  168780  6.2     16.8     15.5  13.3  13.0   
4   20699   18413   14349   24697   20174  6.7     16.2     16.2  14.4  11.3   

   P50_65  PO65  
0    21.1  17.6  
1    20.2  17.3  
2    21.3  18.9  
3    19.6  15.6  
4    19.4  15.8

Before joining, look at the older file and add a tag to the column names - the groups are the same so this will allow us to tell them apart when we calculate our time series.

In [27]:
old.head(3)

CBSA           GEOID   total     U5  \
0         Homosassa Springs, FL Metro Area  310M200US26140  139771   5432   
1  Hickory-Lenoir-Morganton, NC Metro Area  310M200US25860  363936  20102   
2                     Hobbs, NM Micro Area  310M200US26020   66876   5790   

   school  18_20s    30s    40s  50_65    O65  PU5  Pschool  P18_20s  P30s  \
0   16056   13331  10585  14630  32784  46953  3.9     11.5      9.5   7.6   
1   60758   50605  43272  53505  76741  58953  5.5     16.7     13.9  11.9   
2   14142   11683   9115   7799  11186   7161  8.7     21.1     17.5  13.6   

   P40s  P50_65  PO65  
0  10.5    23.5  33.6  
1  14.7    21.1  16.2  
2  11.7    16.7  10.7

In [28]:
#for loop for renaming them with an O* for old at the beginning of each column
for col in old.columns:
    old.rename(columns = {col:'O*'+col}, inplace = True)

In [29]:
#check that this was effective
old.head(3)

O*CBSA         O*GEOID  O*total   O*U5  \
0         Homosassa Springs, FL Metro Area  310M200US26140   139771   5432   
1  Hickory-Lenoir-Morganton, NC Metro Area  310M200US25860   363936  20102   
2                     Hobbs, NM Micro Area  310M200US26020    66876   5790   

   O*school  O*18_20s  O*30s  O*40s  O*50_65  O*O65  O*PU5  O*Pschool  \
0     16056     13331  10585  14630    32784  46953    3.9       11.5   
1     60758     50605  43272  53505    76741  58953    5.5       16.7   
2     14142     11683   9115   7799    11186   7161    8.7       21.1   

   O*P18_20s  O*P30s  O*P40s  O*P50_65  O*PO65  
0        9.5     7.6    10.5      23.5    33.6  
1       13.9    11.9    14.7      21.1    16.2  
2       17.5    13.6    11.7      16.7    10.7

In [30]:
#rename the O*GEOID to the reverted column name to join on it
old = old.rename(columns = {'O*GEOID':'GEOID', 'O*CBSA':'CBSA'})

Now we can join the old df onto the new geodataframe

In [31]:
new['GEOID'] = new['GEOID'].astype(str)
old['GEOID'] = old['GEOID'].astype(str)
new['CBSA'] = new['CBSA'].astype(str)
old['CBSA'] = old['CBSA'].astype(str)

In [32]:
cbsa = pd.merge(new, old, on='CBSA', how = 'inner')

In [33]:
cbsa.head()

CBSA         GEOID_x    total     U5  school  \
0      Big Stone Gap, VA Micro Area  310M500US13720    42456   2214    6189   
1           Billings, MT Metro Area  310M500US13740   179071  10811   30624   
2         Binghamton, NY Metro Area  310M500US13780   241874  12580   35319   
3  Birmingham-Hoover, AL Metro Area  310M500US13820  1085330  67638  182860   
4           Bismarck, ND Metro Area  310M500US13900   127503   8528   20643   

   18_20s     30s     40s   50_65     O65  ...   O*40s  O*50_65   O*O65  \
0    6933    5337    5350    8976    7457  ...    8379    12926    9324   
1   25857   23260   21387   36130   31002  ...   20400    34472   24642   
2   44875   26009   25965   51415   45711  ...   30782    53366   42140   
3  168169  144073  140593  213217  168780  ...  155983   227967  155901   
4   20699   18413   14349   24697   20174  ...   14746    24346   16893   

   O*PU5  O*Pschool  O*P18_20s  O*P30s O*P40s  O*P50_65  O*PO65  
0    5.3       15.1       16.1    12.8   13.8      21.4    15.4  
1    6.5       16.8       15.2    12.7   12.5      21.2    15.1  
2    5.2       15.2       18.3    10.7   12.3      21.4    16.9  
3    6.5       17.1       15.7    13.2   13.7      20.1    13.7  
4    6.7       16.5       17.3    13.2   12.2      20.2    14.0  

[5 rows x 33 columns]

In [34]:
cbsa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 859 entries, 0 to 858
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CBSA       859 non-null    object 
 1   GEOID_x    859 non-null    object 
 2   total      859 non-null    int64  
 3   U5         859 non-null    int64  
 4   school     859 non-null    int64  
 5   18_20s     859 non-null    int64  
 6   30s        859 non-null    int64  
 7   40s        859 non-null    int64  
 8   50_65      859 non-null    int64  
 9   O65        859 non-null    int64  
 10  PU5        859 non-null    float64
 11  Pschool    859 non-null    float64
 12  P18_20s    859 non-null    float64
 13  P30s       859 non-null    float64
 14  P40s       859 non-null    float64
 15  P50_65     859 non-null    float64
 16  PO65       859 non-null    float64
 17  GEOID_y    859 non-null    object 
 18  O*total    859 non-null    int64  
 19  O*U5       859 non-null    int64  
 20  O*school  

# Now we start thinking about the kinds of change we want to look at.

With the measures I think the two most sensical ways to look at things are: percent change in the raw numbers, and change in population share.

## Percent change, limited to 65+, u18, and the rest as one group aka "tax base", I'm dropping the gender aspect here.

First add up the tax base group:

In [35]:
cbsa['O*child'] = cbsa['O*U5']+cbsa['O*school']
cbsa['child'] = cbsa['U5']+cbsa['school']
cbsa['O*taxbase'] = cbsa['O*18_20s']+cbsa['O*30s']+cbsa['O*40s']+cbsa['O*50_65']
cbsa['taxbase'] = cbsa['18_20s']+cbsa['30s']+cbsa['40s']+cbsa['50_65']

In [43]:
cbsa['elderlypercchange'] = round((cbsa['O65'] - cbsa['O*O65'])*100/cbsa['O*O65'], 2)
cbsa['taxbasepercchange'] = round((cbsa['taxbase'] - cbsa['O*taxbase'])*100/cbsa['O*taxbase'], 2)
cbsa['childpercchange'] = round((cbsa['child'] - cbsa['O*child'])*100/cbsa['O*child'], 2)

## Change in population share for our same three groups

Probably move these calculations to the first files and just export groups eventually for the entire project.... gonna finish this for now brain melting.

In [50]:
cbsa['O*Pchild'] = cbsa['O*U5'] + cbsa['O*school']/cbsa['O*total']
cbsa['Pchild'] = cbsa['U5'] + cbsa['school']/cbsa['total']
cbsa['O*Ptaxbase'] = cbsa['O*18_20s']+cbsa['O*30s']+cbsa['O*40s']+cbsa['O*50_65']*100/cbsa['O*total']
cbsa['Ptaxbase'] = cbsa['18_20s']+cbsa['30s']+cbsa['40s']+cbsa['50_65']*100/cbsa['total']

cbsa['elderlysharechange'] = cbsa['PO65'] - cbsa['O*PO65']
cbsa['tbsharechange'] = cbsa['Ptaxbase'] - cbsa['O*Ptaxbase']
cbsa['childsharechange'] = cbsa['Pchild'] - cbsa['O*Pchild']
cbsa['childsharechange'] = cbsa['Pchild'] - cbsa['O*Pchild']

In [51]:
#make another small dataframe to check out high and low values, who's around the Nashville MSA... etc.
groupperc = cbsa[['CBSA', 'GEOID_x', 'elderlypercchange',
                  'taxbasepercchange', 'childpercchange', 
                  'elderlysharechange', 'tbsharechange', 'childsharechange']]

In [52]:
groupperc.head()

CBSA         GEOID_x  elderlypercchange  \
0      Big Stone Gap, VA Micro Area  310M500US13720             -20.02   
1           Billings, MT Metro Area  310M500US13740              25.81   
2         Binghamton, NY Metro Area  310M500US13780               8.47   
3  Birmingham-Hoover, AL Metro Area  310M500US13820               8.26   
4           Bismarck, ND Metro Area  310M500US13900              19.42   

   taxbasepercchange  childpercchange  elderlysharechange  tbsharechange  \
0             -31.45           -32.10                 2.2   -8254.223754   
1               6.34             9.11                 2.2    4699.013867   
2              -5.16            -5.71                 2.0   -6108.152752   
3              -6.46            -6.40                 1.9  -31217.430392   
4               3.07             4.31                 1.8    1977.197064   

   childsharechange  
0      -1005.005567  
1        191.003089  
2       -295.006119  
3      -5659.002641  
4        470.996940

In [48]:
#index into where the Nashville MSA is to check the rankings out
nash = groupperc.loc[groupperc['CBSA'] == 'Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area'].reset_index(drop = True)

In [49]:
print("The following number is Nashville's percent growth in a 5 year over 5 year period for total elderly population:")
print('______________________________________________________________________________')
print(nash['elderlypercchange'])
print("The following number is Nashville's percent growth in a 5 year over 5 year period for total tax base population:")
print('______________________________________________________________________________')
print(nash['taxbasepercchange'])
print("The following number is Nashville's percent growth in a 5 year over 5 year period for total child population:")
print('______________________________________________________________________________')
print(nash['childpercchange'])

The following number is Nashville's percent growth in a 5 year over 5 year period for total elderly population:
______________________________________________________________________________
0    21.72
Name: elderlypercchange, dtype: float64
The following number is Nashville's percent growth in a 5 year over 5 year period for total tax base population:
______________________________________________________________________________
0    6.97
Name: taxbasepercchange, dtype: float64
The following number is Nashville's percent growth in a 5 year over 5 year period for total child population:
______________________________________________________________________________
0    4.9
Name: childpercchange, dtype: float64


In [ ]:
#make another small dataframe to check out high and low values, who's around the Nashville MSA... etc.
sharechange = cbsa[['FullName', 'CBSAFP', 'elderlysharechange',
                    'tbsharechange', 'childsharechange']]

In [ ]:
#create a list of the columns you want to rank and then create a for loop to write in the rankings as integers
cols = ['elderlysharechange','tbsharechange','childsharechange']

for i in cols:
    sharechange['rank{}.'.format(i)] = sharechange['{}'.format(i)].rank().astype(int)
    
sharechange = sharechange.copy()

In [ ]:
#index into where the Nashville MSA is to check the rankings out
nash = sharechange.loc[sharechange['CBSAFP'] == 34980].reset_index(drop = True)

In [ ]:
print("The following number is Nashville's rank in total population share change in a 5 year over 5 year period for total elderly population:")
print('______________________________________________________________________________')
print(nash['rankelderlysharechange.'])
print(nash['elderlysharechange'])
print("The following number is Nashville's rank in total population share change in a 5 year over 5 year period for total tax base population:")
print('______________________________________________________________________________')
print(nash['ranktbsharechange.'])
print(nash['tbsharechange'])
print("The following number is Nashville's rank in total population share change in a 5 year over 5 year period for total child population:")
print('______________________________________________________________________________')
print(nash['rankchildsharechange.'])
print(nash['childsharechange'])